## Restaurant Recommendation

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [2]:
dataf=pd.read_csv("dataset.csv")
dataf.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3.0,4.8,Dark Green,Excellent,314.0
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3.0,4.5,Dark Green,Excellent,591.0
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4.0,4.4,Green,Very Good,270.0
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4.0,4.9,Dark Green,Excellent,365.0
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4.0,4.8,Dark Green,Excellent,229.0


In [3]:
dataf=dataf.dropna()
dataf.isna().sum()

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                0
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64

In [4]:
# Step 1: Ensure all relevant columns are strings to avoid the .str accessor error
dataf['Cuisines'] = dataf['Cuisines'].astype(str)
dataf['Price range'] = dataf['Price range'].astype(str)
dataf['City'] = dataf['City'].astype(str)
dataf['Has Online delivery'] = dataf['Has Online delivery'].astype(str)

In [5]:
# Check if any of these columns contain NaN values and fill them with empty strings
dataf['Cuisines'] = dataf['Cuisines'].fillna('')
dataf['Price range'] = dataf['Price range'].fillna('')
dataf['City'] = dataf['City'].fillna('')
dataf['Has Online delivery'] = dataf['Has Online delivery'].fillna('')

In [6]:
# Step 2: Combine relevant features into a single string for recommendation
dataf['combined_features'] = dataf['Cuisines'] + ' ' + dataf['Price range'] + ' ' + dataf['City'] + ' ' + dataf['Has Online delivery']

In [7]:
# Step 3: Normalize the 'Has Online delivery' column to 'yes' and 'no'
dataf['Has Online delivery'] = dataf['Has Online delivery'].apply(lambda x: 'yes' if str(x).lower() in ['yes', '1', 'true'] else 'no')


In [8]:
# Step 4: Recommendation function (same as before)
def recommend_restaurants(dataf, cuisine, price_range, min_rating, city_name, online_delivery, num_recommendations=5):
    print("Initial Data Size: ", len(dataf))
    
    # Step 4a: Filter based on minimum rating
    filtered_df = dataf[dataf['Aggregate rating'] >= min_rating]
    print("After Rating Filter: ", len(filtered_df))
    if filtered_df.empty:
        return "No restaurants found that match the given preferences."

    # Step 4b: Filter based on city
    filtered_df = filtered_df[filtered_df['City'].str.contains(city_name.lower(), case=False)]
    print("After City Filter: ", len(filtered_df))
    if filtered_df.empty:
        return "No restaurants found that match the given preferences."

    # Step 4c: Filter based on online delivery availability
    filtered_df = filtered_df[filtered_df['Has Online delivery'] == online_delivery.lower()]
    print("After Online Delivery Filter: ", len(filtered_df))
    if filtered_df.empty:
        return "No restaurants found that match the given preferences."

    # Step 4d: Filter based on price range
    filtered_df = filtered_df[filtered_df['Price range'].str.contains(price_range.lower(), case=False)]
    print("After Price Range Filter: ", len(filtered_df))
    if filtered_df.empty:
        return "No restaurants found that match the given preferences."

    # Step 4e: Filter based on cuisine (partial string match)
    cuisines_str = '|'.join(cuisine).lower()
    filtered_df = filtered_df[filtered_df['Cuisines'].str.contains(cuisines_str, case=False, na=False)]
    print("After Cuisine Filter: ", len(filtered_df))
    if filtered_df.empty:
        return "No restaurants found that match the given preferences."

    # Step 5: Combine features to calculate cosine similarity
    user_preference_str = ' '.join(cuisine).lower() + f" {price_range.lower()} {online_delivery.lower()} {city_name.lower()}"
    
    # Apply CountVectorizer
    vectorizer = CountVectorizer()
    
    # Fit transform on the combined features of restaurants
    combined_feature_mtx = vectorizer.fit_transform(filtered_df['combined_features'].values.astype('U'))

    # Transform the user preference string
    user_preference_matrix = vectorizer.transform([user_preference_str])

    # Compute cosine similarity
    cosine_sim = cosine_similarity(combined_feature_mtx, user_preference_matrix).flatten()

    # Get the indices of the top similar restaurants
    top_indices = cosine_sim.argsort()[-num_recommendations:][::-1]

    if not top_indices.size:
        return "No restaurants found that match the given preferences."

    # Get the top recommendations
    recommendation = filtered_df.iloc[top_indices]

    return recommendation[['Restaurant Name', 'Cuisines', 'Price range', 'Aggregate rating', 'City', 'Has Online delivery']]


In [9]:
# Step 6: Get user input (same as before)
print("Enter your preferences for the restaurant recommendation:")
cuisine_input = [str(x).lower().strip() for x in input("Cuisine (e.g., Chinese, Italian, etc.): ").split(',')]
price_range_input = input("Price Range : ").strip().lower()
min_rating_input = float(input("Minimum Rating (0 to 5): "))
location_input = input("Location (e.g., New York, San Francisco, etc.): ").strip().lower()
online_delivery_input = input("Is Online Delivery Available (Yes/No): ").strip().lower()

# Step 7: Display the recommendations
recommendations = recommend_restaurants(
    dataf=dataf,
    cuisine=cuisine_input,
    price_range=price_range_input,
    min_rating=min_rating_input,
    city_name=location_input,
    online_delivery=online_delivery_input,
    num_recommendations=5
)

print("\nRecommended Restaurants:")
recommendations

Enter your preferences for the restaurant recommendation:


Cuisine (e.g., Chinese, Italian, etc.):  chinese
Price Range :  3
Minimum Rating (0 to 5):  4
Location (e.g., New York, San Francisco, etc.):  delhi
Is Online Delivery Available (Yes/No):  yes


Initial Data Size:  4401
After Rating Filter:  903
After City Filter:  148
After Online Delivery Filter:  65
After Price Range Filter:  28
After Cuisine Filter:  4

Recommended Restaurants:


,Restaurant Name,Cuisines,Price range,Aggregate rating,City,Has Online delivery
3825,Feng Shuii,"Thai, Chinese",3.0,4.3,New Delhi,yes
3699,Noshi - Yum Asian Delivery,"Chinese, Thai, Japanese",3.0,4.2,New Delhi,yes
3266,RoadRomeo,"North Indian, Hyderabadi, Kashmiri, Chinese",3.0,4.0,New Delhi,yes
3518,Asian Haus,"Chinese, Thai, Asian, Malaysian, Vietnamese, J...",3.0,4.1,New Delhi,yes
